<a href="https://colab.research.google.com/github/OctavaOrigin/Deep-Learning/blob/main/Dota_matches_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import json
import numpy as np

In [2]:
from collections import Counter

In [3]:
!pip install pyopendota

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=f025aa965d131c9b024c2e4a936ac9046e091d25054f165a9f4fda552c041648
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


In [4]:
import opendota

# Initialize the API-connection object
client = opendota.OpenDota()

In [5]:
df = pd.DataFrame.from_dict(client.get_pro_players(), orient='columns')

In [6]:
# leagues = pd.DataFrame(client.get_leagues())

In [7]:
# leagues.loc[leagues['tier'] == 'premium']

In [8]:
# pd.DataFrame(client.get_league_matches(9870))

In [9]:
df = df.loc[df['team_name'].isin(['Gaimin Gladiators', 'Team Liquid', 'Tundra Esports', 'BetBoom Team', '9Pandas', 'Evil Geniuses', 'PSG.LGD', 'Shopify Rebellion', 'Talon', 'beastcoast', 'Team Spirit', 'TSM', 'Team Aster', 'Quest Esports', 'Team Secret', 'Entity', 'Virtus.pro', 'Nigma Galaxy', 'Xtreme Gaming', 'Execration', 'BLEED', 'nouns', 'INVICTUS GAMING', 'HYDRA', 'Alliance', 'Old G', 'B8', 'One Move', 'Team SMG', 'Azure Ray'])]

In [10]:
df.team_name.unique()

array(['Team SMG', 'Team Secret', 'Tundra Esports', 'Gaimin Gladiators',
       'Alliance', 'Shopify Rebellion', 'Evil Geniuses', 'Nigma Galaxy',
       'TSM', 'nouns', 'Team Liquid', 'B8', 'HYDRA', 'BLEED', 'Old G',
       'One Move', 'Talon', 'Team Aster', 'Virtus.pro', 'Azure Ray',
       'Quest Esports', 'Entity', 'INVICTUS GAMING', 'Team Spirit',
       'Execration', 'PSG.LGD', 'beastcoast', 'BetBoom Team', '9Pandas',
       'Xtreme Gaming'], dtype=object)

In [11]:
# df.loc[df['name'] == 'dyrachyo']

In [12]:
# df.columns

In [13]:
# data = pd.DataFrame.from_dict(client.get_player_matches(player_id = 116934015, days = None), orient='columns')
# data

In [14]:
def swap_values(player_slot, hero_id, playerId, team_id):
  combined_lists = list(zip(player_slot, hero_id, playerId, team_id))
  combined_lists.sort(key=lambda x: x[0])
  player_slot, hero_id, playerId, team_id = zip(*combined_lists)
  return player_slot, hero_id, playerId, team_id

In [15]:
def devide_team_columns(team_id):
  team_id_1 = team_id[:5]
  team_id_2 = team_id[-5:]

  counter1 = Counter(team_id_1)
  counter2 = Counter(team_id_2)

  mcv1 = counter1.most_common(1)[0][0]
  mcv2 = counter2.most_common(1)[0][0]

  return mcv1, mcv2

In [16]:
def add_patch(match_id):
  a = client.get_match(match_id)
  return a['patch']

In [17]:
def func(row_value, team_id):
    try:
        daf = pd.DataFrame.from_dict(client.get_player_matches(player_id=row_value), orient='columns')
        daf = daf.loc[:, ['match_id', 'start_time', 'duration', 'player_slot', 'radiant_win', 'hero_id']]
        daf['playerId'] = row_value
        daf['team_id'] = team_id
        return daf
    except Exception as e:
        print(f"Error occurred for row_value: {row_value}. Skipping this cycle.")
        return None

In [18]:
def pid(row):
  for i in row['hero_id']:
    row[str(i)] = 1
  # for i in row['playerId']:
  #   row[str(i)] = 1

  row[str(row['team_id_1'])] = 1
  row[str(row['team_id_2'])] = 1

  return row

Preprocessing

In [21]:
list_of_dfs = df.apply(lambda row: func(row['account_id'], row['team_id']), axis=1)
stacked_df = pd.concat(list_of_dfs.tolist(), ignore_index=True)

Error occurred for row_value: 6922000. Skipping this cycle.
Error occurred for row_value: 26771994. Skipping this cycle.
Error occurred for row_value: 50828662. Skipping this cycle.
Error occurred for row_value: 72312627. Skipping this cycle.
Error occurred for row_value: 108831569. Skipping this cycle.
Error occurred for row_value: 121769650. Skipping this cycle.
Error occurred for row_value: 140153524. Skipping this cycle.
Error occurred for row_value: 165671428. Skipping this cycle.
Error occurred for row_value: 166458146. Skipping this cycle.
Error occurred for row_value: 168340162. Skipping this cycle.
Error occurred for row_value: 177453057. Skipping this cycle.
Error occurred for row_value: 187758589. Skipping this cycle.
Error occurred for row_value: 192914280. Skipping this cycle.
Error occurred for row_value: 330191088. Skipping this cycle.
Error occurred for row_value: 898754153. Skipping this cycle.


In [22]:
data = stacked_df.copy()
data = stacked_df.loc[:, ['match_id', 'start_time', 'duration', 'player_slot', 'radiant_win', 'hero_id', 'playerId', 'team_id']]
data = data.loc[~pd.isna(data['radiant_win'])]
data['player_slot'] = data['player_slot'].apply(lambda x: 0 if x <= 127 else 1)
data['radiant_win'] = data['radiant_win'].astype('int')

data = data.groupby(['match_id', 'start_time', 'duration', 'radiant_win']).agg(lambda x: list(x)).reset_index()
data = data.loc[data['hero_id'].apply(lambda x: len(x) == 10)]
data[['player_slot', 'hero_id', 'playerId', 'team_id']] = data.apply(lambda row: swap_values(row['player_slot'], row['hero_id'], row['playerId'], row['team_id']), axis=1, result_type='expand')

data[['team_id_1', 'team_id_2']] = data.apply(lambda row: devide_team_columns(row['team_id']), axis=1, result_type='expand')
# data.drop(['player_slot', 'hero_id', 'playerId', 'team_id'], axis=1, inplace=True)

# data['patch'] = data.apply(lambda row: add_patch(row['match_id']), axis=1, result_type='expand')
data.drop(['player_slot', 'team_id', 'duration', 'match_id'], axis=1, inplace=True)

In [23]:
data.drop(['playerId'], axis=1, inplace=True)

In [24]:
combined_playerId_values = []
combined_hero_id = []
teams = []

# for row in data['playerId']:
#   combined_playerId_values.extend(row)

for row in data['hero_id']:
  combined_hero_id.extend(row)

teams = data['team_id_1'].to_list()
teams += data['team_id_2'].to_list()

# cpv_set = set(combined_playerId_values)
chi_set = set(combined_hero_id)
teams_set = set(teams)

# zeros_cpv_df = pd.DataFrame(0, index=data.index, columns=[f'{i}' for i in cpv_set])
zeros_chi_df = pd.DataFrame(0, index=data.index, columns=[f'{i}' for i in chi_set])
zeros_teams_df = pd.DataFrame(0, index=data.index, columns=[f'{i}' for i in teams_set])
# zeros = pd.concat([zeros_chi_df, zeros_cpv_df], axis=1)

data = pd.concat([data, zeros_teams_df], axis=1)
data = pd.concat([data, zeros_chi_df], axis=1)

data = data.apply(pid, axis=1)
data.drop(['hero_id', 'team_id_1', 'team_id_2'], axis=1, inplace=True)

In [25]:
data

,start_time,radiant_win,5,7554697,8255756,8244493,6209166,15,8255888,7119388,...,120,121,123,126,128,129,135,136,137,138
262,1679317037,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
286,1679322179,1,0,1,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
311,1679325778,1,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
377,1679344492,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
386,1679348138,1,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41040,1694682554,1,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
41052,1694685774,1,0,0,1,0,0,0,0,0,...,0,1,0,0,1,1,0,0,0,0
41081,1694691267,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
41162,1694705460,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [26]:
target = data['radiant_win']
data.drop(['radiant_win'], axis=1, inplace=True)

In [27]:
from sklearn.preprocessing import StandardScaler

In [28]:
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

In [29]:
data_scaled = pd.DataFrame(data_scaled)

In [30]:
data_scaled.head()

,0,1,2,3,4,5,6,7,8,9,...,144,145,146,147,148,149,150,151,152,153
0,-1.756588,-0.359434,4.997142,-0.393725,-0.110677,-0.191018,-0.408510,-0.384353,-0.351581,-0.446328,...,-0.534522,-0.149991,-0.164677,-0.384353,1.984998,-0.310647,-0.184742,-0.208873,-0.351581,-0.171596
1,-1.755286,-0.359434,4.997142,-0.393725,-0.110677,-0.191018,2.447919,-0.384353,-0.351581,-0.446328,...,-0.534522,-0.149991,-0.164677,-0.384353,1.984998,-0.310647,-0.184742,-0.208873,-0.351581,-0.171596
2,-1.754375,-0.359434,4.997142,-0.393725,-0.110677,-0.191018,-0.408510,-0.384353,-0.351581,-0.446328,...,-0.534522,-0.149991,6.072479,-0.384353,-0.503779,-0.310647,-0.184742,-0.208873,-0.351581,-0.171596
3,-1.749637,-0.359434,-0.200114,-0.393725,-0.110677,-0.191018,-0.408510,-0.384353,-0.351581,-0.446328,...,-0.534522,-0.149991,-0.164677,-0.384353,-0.503779,-0.310647,-0.184742,-0.208873,-0.351581,-0.171596
4,-1.748714,-0.359434,-0.200114,-0.393725,-0.110677,-0.191018,-0.408510,-0.384353,-0.351581,-0.446328,...,1.870829,-0.149991,-0.164677,-0.384353,1.984998,-0.310647,-0.184742,-0.208873,-0.351581,-0.171596


In [31]:
from sklearn.decomposition import PCA

In [32]:
pca = PCA()
pca.fit(data_scaled)

PCA()

In [33]:
cumulative_explained_variance = np.cumsum(pca.explained_variance_ratio_)

# Determine the number of components needed for 99% explained variance
n_components = np.argmax(cumulative_explained_variance >= 0.90) + 1

# Select the first n_components principal components
selected_components = pca.components_[:n_components]

# You can transform your data to the selected components
data_reduced = pca.transform(data_scaled)[:, :100]

In [34]:
n_components

113

In [35]:
pd.DataFrame(data_reduced)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.797494,1.865455,-2.563418,-1.756621,-0.848311,-0.635545,-1.591504,-0.221336,-1.888797,0.771093,...,-0.825844,-0.007534,-0.726218,-0.753872,-0.137437,1.786849,0.000441,-0.016201,-0.178660,0.685986
1,0.789699,2.343799,-0.477439,-4.239762,-1.279931,0.360655,-3.075778,-0.561829,1.761880,2.298201,...,-1.536896,0.017871,-0.439704,-2.648848,-0.678756,-0.879185,1.978544,0.636560,-0.319967,-0.838880
2,2.884183,2.533987,-1.905579,-1.078584,-1.247077,0.819055,-1.478870,0.769081,-0.357097,0.641147,...,-1.003947,-0.025041,-0.985244,0.992035,-0.530484,1.444957,-0.530454,0.239656,-1.553222,1.044557
3,3.317395,3.782399,-2.855663,-0.722612,-2.403089,0.746342,-3.951955,0.664642,-1.058565,-3.298728,...,-1.705998,0.336790,1.980398,-0.372574,-0.011790,-0.010346,-0.360318,0.463030,0.057733,-0.184474
4,1.706873,2.159067,-1.705620,-0.102625,-1.859266,1.119723,-2.600932,-0.355279,-0.348303,-2.283356,...,0.768382,-0.587702,-0.396488,-1.021860,-0.427384,0.136002,-1.148804,-0.514922,-0.415181,-0.772171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,-1.329508,2.291420,0.025128,0.827062,-0.953800,-0.263600,0.297081,-0.729678,-3.737570,0.772474,...,0.329855,-0.928810,0.418999,-0.832986,0.612780,-0.074193,-0.591761,-1.004002,1.443121,0.802817
905,-2.465374,4.081999,1.705073,1.354844,-1.093481,-1.304313,1.633393,0.777399,-1.631168,1.853574,...,-0.075438,-0.497210,0.713489,0.090363,2.559024,0.689912,0.076577,0.760931,0.016316,0.435512
906,-2.237978,3.245229,1.164152,1.001779,-1.086120,-0.668915,0.521662,-0.847114,-2.345989,0.936069,...,-0.763435,-0.339010,1.069811,-0.797302,1.176676,-1.482077,0.289862,-0.397161,-1.693856,-0.287380
907,-2.948769,3.152981,2.067445,5.041127,-1.035442,-0.487088,-1.454835,-1.912120,2.334233,-1.921362,...,0.761892,-1.178115,0.803744,0.645157,-1.044362,-1.005776,-1.694997,-0.314030,0.445856,0.120093


In [39]:
data_reshaped = data_reduced.reshape((909, 10, 10, 1))

Preprocessing end

In [42]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [43]:
model = Sequential()

model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(10, 10, 1)))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(16, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [46]:
from sklearn.model_selection import train_test_split

In [47]:
X_train, X_test, y_train, y_test = train_test_split(data_reshaped, target, random_state = 42)

In [ ]:
model.fit(X_train, y_train, epochs=20, batch_size=32)

In [51]:
y_pred = model.predict(X_test)

8/8 [==============================] - 0s 3ms/step


In [52]:
from sklearn.metrics import accuracy_score

In [58]:
accuracy_score(y_test, (y_pred > 0.5).astype(int))

0.5219298245614035

Данных слишком мало.